In [1]:
%pwd

'd:\\Kidney-Disease-Classification-Project\\research'

In [2]:
import os

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Kidney-Disease-Classification-Project'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/JAYANTHNITW/Kidney-Disease-Classification-Project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="JAYANTHNITW"
os.environ["MLFLOW_TRACKING_PASSWORD"]="9e70361d6094119609e2599caf804a1365ab101d"

In [5]:
import tensorflow as tf

In [6]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_url: str
    params_image_size: list
    params_batch_size: int

In [8]:
from src.CNNclassifier.constants import *
from src.CNNclassifier.utils.common import read_yaml,create_directories,save_json

In [9]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/CT-scan_kidney_normal_tumor_images",
            mlflow_url="https://dagshub.com/JAYANTHNITW/Kidney-Disease-Classification-Project.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        
        return eval_config

In [10]:
from src.CNNclassifier import logger
import gdown,zipfile,time
import tensorflow as tf
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [11]:
class Evaluation:
    def __init__(self,config: EvaluationConfig):
        self.config = config

    def train_valid_generator(self):

        datagenerator_kwargs = dict(rescale = 1./255,
                                    validation_split = 0.20)
        
        dataflow_kwargs = dict(target_size = self.config.params_image_size[:-1],
                               #batch_size = self.config.params_batch_size,
                               interpolation="bilinear")
        
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )
        
    @staticmethod
    def load_model(path:Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self.train_valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss":self.score[0],"accuracy":self.score[1]}
        save_json(path=Path("scores.json"),data=scores)


    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_url)
        tracking_url_type_store=urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss":self.score[0],"accuracy":self.score[1]}
            )

            #Model registry does not work with file store
            if tracking_url_type_store != "file":
                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [12]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2023-12-31 17:09:22,786: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-31 17:09:22,792: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-31 17:09:22,796: INFO: common: created directory at: artifacts]


Found 300 images belonging to 2 classes.

[2023-12-31 17:09:25,629: WARNING: module_wrapper: From c:\Users\DELL\anaconda3\Lib\site-packages\keras\src\utils\tf_utils.py:492: The name tf.ragged.RaggedTensorValue is deprecated. Please use tf.compat.v1.ragged.RaggedTensorValue instead.
]
10/10 [==============================] - 84s 8s/step - loss: 0.7224 - accuracy: 0.7667
[2023-12-31 17:10:48,843: INFO: common: json file saved at : scores.json]


2023/12/31 17:10:49 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\DELL\AppData\Local\Temp\tmpulngk8_0\model\data\model\assets
[2023-12-31 17:10:55,483: INFO: builder_impl: Assets written to: C:\Users\DELL\AppData\Local\Temp\tmpulngk8_0\model\data\model\assets]


c:\Users\DELL\anaconda3\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
